            DATA ANALYSIS FOR PHARMOUTICAL DATA 

In [3]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import warnings



In [4]:
logging.basicConfig(level=logging.INFO, filename='../log/data_exploration.log', 
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [7]:
train_data = pd.read_csv('../data/train.csv', low_memory=False)
test_data = pd.read_csv('../data/test.csv', low_memory=False)
store_data = pd.read_csv('../data/store.csv', low_memory=False)

logging.info("Data Loaded: Train data shape: {}, Test data shape: {}".format(train_data.shape, test_data.shape))


In [8]:
missing_values_train = train_data.isnull().sum()
missing_values_test = test_data.isnull().sum()

logging.info("Missing values in train data:\n{}".format(missing_values_train[missing_values_train > 0]))
logging.info("Missing values in test data:\n{}".format(missing_values_test[missing_values_test > 0]))


In [9]:
train_data.dropna(inplace=True)

In [10]:
Q1 = train_data['Sales'].quantile(0.25)
Q3 = train_data['Sales'].quantile(0.75)
IQR = Q3 - Q1
outliers = train_data[(train_data['Sales'] < (Q1 - 1.5 * IQR)) | (train_data['Sales'] > (Q3 + 1.5 * IQR))]

logging.info("Number of outliers detected: {}".format(outliers.shape[0]))

